### Julia installation Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.11.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.11.1 on the current Colab Runtime...
2024-12-11 03:20:00 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.11/julia-1.11.1-linux-x86_64.tar.gz [254553793/254553793] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.11

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [2]:
using Pkg
Pkg.add("Latexify")
Pkg.add("LaTeXStrings")
Pkg.add("Combinatorics")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Format ───────────── v1.3.7
   Installed Latexify ─────────── v0.16.5
   Installed LaTeXStrings ─────── v1.4.0
   Installed OrderedCollections ─ v1.7.0
   Installed MacroTools ───────── v0.5.13
   Installed Requires ─────────── v1.3.0
    Updating `~/.julia/environments/v1.11/Project.toml`
  [23fbe1c1] + Latexify v0.16.5
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [1fa38f19] + Format v1.3.7
  [b964fa9f] + LaTeXStrings v1.4.0
  [23fbe1c1] + Latexify v0.16.5
  [1914dd2f] + MacroTools v0.5.13
  [bac558e1] + OrderedCollections v1.7.0
  [ae029012] + Requires v1.3.0
Precompiling project...
   2731.0 ms  ✓ LaTeXStrings
   3685.5 ms  ✓ OrderedCollections
   2404.3 ms  ✓ Requires
   5924.5 ms  ✓ Format
   6546.5 ms  ✓ MacroTools
   4712.9 ms  ✓ Latexify
  6 dependencies successfully precompiled in 16 seconds. 47 already precompiled.
   Resolving package versions...
    Updati

# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

## Source code

In [1]:
module LogToolKit



export
    Symb, ISymb, HSymb, monomial, lincomb, polynomial, tensor, Phi, Phi_inv, fundamental_column,
    ordered_partitions



using LaTeXStrings
using Latexify
using Combinatorics



function ordered_partitions(n::Int, k::Int; positive=true)
    if positive
        return union((unique(permutations(unordered_partition)) for unordered_partition in partitions(n + k, k))...)
    else
        return [partition .- 1 for partition in ordered_partitions(n, k)]
    end
end



abstract type Symb end



struct monomial{T}
    # T must be comparable type that has the latex_repr attribute
    args::Tuple{Vararg{Pair{T, Int}}}
    deg::Int
    latex_repr::String
end

# constructor for monomial{T}, Int refers to the exponent
function monomial{T}(args::Vararg{Union{Tuple{T, Int}, Pair{T, Int}, T}}) where T
    # multiply the variables
    counter = Dict{T, Int}()
    for var in args
        (key, value) = typeof(var) <: T ? (var, 1) : var
        counter[key] = get(counter, key, 0) + value
    end

    # sort the variables
    args = Tuple(sort(filter(x->x.second!=0, collect(counter))))

    # return 1 when the args is empty
    if length(args) == 0
        return 1
    end

    # compute the degree of the monomial
    deg = sum([exponent for (_, exponent) in args])

    # generate latex_repr
    latex_repr = ""
    for (var, exponent) in args
        if exponent == 1
            latex_repr *= "$(var.latex_repr)"
        else
            latex_repr *= "$(var.latex_repr)^{$(exponent)}"
        end
    end

    return monomial{T}(args, deg, latex_repr)
end

struct lincomb{T}
    # T must be comparable type that has the latex_repr attribute
    args::Tuple{Vararg{Pair{T, Number}}}
    intercept::Number
    latex_repr::String
end
polynomial{T} = lincomb{Union{T, monomial{T}}}

# constructor for lincomb, Number refers to the coefficient
# Number is in the second argument to ease the sorting of variables
function lincomb{T}(args::Vararg{Union{T, Tuple{T, Number}, Pair{T, Number}}}; intercept=0) where T
    # add the variables
    counter = Dict{T, Number}()
    for term in args
        (key, value) = typeof(term) <: T ? (term, 1) : term
        counter[key] = get(counter, key, 0) + value
    end

    # sort the variables
    args = Tuple(sort(filter(x->x.second!=0, collect(counter))))

    # return the intercept when the args is empty
    if length(args) == 0
        return intercept
    end

    # generate latex_repr
    argstr = String[]
    if intercept != 0
        push!(argstr, "($intercept)")
    end
    for term in args
        if term.second == 1
            push!(argstr, term.first.latex_repr)
        else
            push!(argstr, "($(term.second))" * term.first.latex_repr)
        end
    end
    latex_repr = join(argstr, '+')

    return lincomb{T}(args, intercept, latex_repr)
end

struct tensor{T}
    # T must be a type that has the latex_repr attribute
    args::Tuple{Vararg{T}}
    latex_repr::String
end
tensors{T} = lincomb{Union{T, tensor{T}}}

# constructor for tensor{T}
function tensor{T}(args::Vararg{T}) where T
    if length(args) == 0
        return 1
    elseif length(args) == 1
        return args[1]
    else
        return tensor{T}(args, join(map(x -> x.latex_repr, args), "\\otimes "))
    end
end



## Overloading the Base operators ==, <, *, +, -, inv, ^


Base.isequal(a::Number, b::Union{T, monomial{T}, polynomial{T}, tensor{T}}) where {T <: Symb} = false
Base.isequal(a::T, b::monomial{T}) where {T <: Symb} = monomial{T}(a) == b
Base.isequal(a::Union{T, monomial{T}}, b::polynomial{T}) where {T <: Symb} = polynomial{T}(a) == b


Base.isless(a::Number, b::Union{T, monomial{T}, polynomial{T}, tensor{T}}) where {T <: Symb} = true
Base.isless(a::Union{T, monomial{T}, polynomial{T}, tensor{T}}, b::Number) where {T <: Symb} = false
Base.isless(a::monomial{T}, b::monomial{T}) where {T <: Symb} = (a.deg, reverse(a.args)) < (b.deg, reverse(b.args))
Base.isless(a::polynomial{T}, b::polynomial{T}) where {T <: Symb} = (reverse(a.args), a.intercept) < (reverse(b.args), b.intercept)
Base.isless(a::T, b::monomial{T}) where {T <: Symb} = monomial{T}(a) < b
Base.isless(a::monomial{T}, b::T) where {T <: Symb} = a < monomial{T}(b)
Base.isequal(a::Union{T, monomial{T}}, b::polynomial{T}) where {T <: Symb} = polynomial{T}(a) < b
Base.isequal(a::polynomial{T}, b::Union{T, monomial{T}}) where {T <: Symb} = a < polynomial{T}(b)
Base.isless(a::tensor{T}, b::tensor{T}) where {T <: Symb} = (length(a), a.args) < (length(b), b.args)


Base.:+(a::Number, b::Union{T, monomial{T}}) where {T <: Symb} = polynomial{T}(b; intercept=a)
Base.:+(a::Number, b::polynomial{T}) where {T <: Symb} = polynomial{T}(b.args...; intercept = a + b.intercept)
Base.:+(a::Union{T, monomial{T}, polynomial{T}}, b::Number) where {T <: Symb} = b + a
Base.:+(a::Union{T, monomial{T}}, b::Union{T, monomial{T}}) where {T <: Symb} = polynomial{T}(a, b)
Base.:+(a::polynomial{T}, b::polynomial{T}) where {T <: Symb} = polynomial{T}(a.args..., b.args...; intercept = a.intercept + b.intercept)
Base.:+(a::Union{T, monomial{T}}, b::polynomial{T}) where {T <: Symb} = polynomial{T}(a, b.args...; intercept = b.intercept)
Base.:+(a::polynomial{T}, b::Union{T, monomial{T}}) where {T <: Symb} = b + a


Base.:-(a::Union{T, monomial{T}}) where {T <: Symb} = polynomial{T}((a, -1))
Base.:-(a::polynomial{T}) where {T <: Symb} = polynomial{T}(map(x->(x.first, -x.second), a.args)...; intercept=-a.intercept)


Base.:-(a::Number, b::Union{T, monomial{T}, polynomial{T}}) where {T <: Symb} = a + (-b)
Base.:-(a::Union{T, monomial{T}, polynomial{T}}, b::Number) where {T <: Symb} = a + (-b)
Base.:-(a::Union{T, monomial{T}, polynomial{T}}, b::Union{T, monomial{T}, polynomial{T}}) where {T <: Symb} = a + (-b)


Base.:*(a::Number, b::Union{T, monomial{T}}) where {T <: Symb} = polynomial{T}((b, a))
Base.:*(a::Number, b::polynomial{T}) where {T <: Symb} = polynomial{T}(map(x->(x.first, a*x.second), b.args)...; intercept=a*b.intercept)
Base.:*(a::Union{T, monomial{T}, polynomial{T}}, b::Number) where {T <: Symb} = b * a
Base.:*(a::Union{T, monomial{T}}, b::Union{T, monomial{T}}) where {T <: Symb} = monomial{T}((a isa T ? [a] : a.args)..., (b isa T ? [b] : b.args)...)
Base.:*(a::Union{T, monomial{T}}, b::polynomial{T}) where {T <: Symb} = polynomial{T}((a, b.intercept), map(x->(a*x.first, x.second), b.args)...)
Base.:*(a::polynomial{T}, b::Union{T, monomial{T}}) where {T <: Symb} = b * a
Base.:*(a::polynomial{T}, b::polynomial{T}) where {T <: Symb} =
    polynomial{T}(
        vec([(t1.first * t2.first, t1.second * t2.second) for t1 in a.args, t2 in b.args])...,
        [(t.first, a.intercept * t.second) for t in b.args]...,
        [(t.first, b.intercept * t.second) for t in a.args]...,
        ; intercept = a.intercept * b.intercept
    )
Base.:*(a::tensor{T}, b::tensor{T}) where {T <: Symb} = tensor(x * y for (x,y) in zip(a.args, b.args))
Base.:*(a::tensor{T}, b::lincomb{tensor{T}}) where {T <: Symb} = lincomb{tensor{T}}((a, b.intercept), map(x->(a*x.first, x.second), b.args)...)
Base.:*(a::lincomb{tensor{T}}, b::tensor{T}) where {T <: Symb} = b * a


Base.inv(a::T) where {T <: Symb} = monomial{T}((a, -1))
Base.:inv(a::monomial{T}) where {T <: Symb} = monomial{T}(map(x -> (x.first, -x.second), a.args)...)


Base.:^(a::T, n::Int) where {T <: Symb} = monomial{T}((a, n))
Base.:^(a::monomial{T}, n::Int) where {T <: Symb} = monomial{T}(map(x -> (x.first, n*x.second), a.args)...)
Base.:^(a::polynomial{T}, n::Int) where {T <: Symb} =
    if n < 0
        throw(ArgumentError("the exponent must be at least 0"))
    elseif n <= 1
        n == 1 ? a : 1
    else
        temp = a^div(n, 2)
        temp * temp * (n % 2 == 0 ? 1 : a)
    end



"""
    ISymb(args::Vararg{Int})

Return an Iterated integral symbol with args
"""

struct ISymb <: Symb
    args::Tuple{Vararg{Int}}
    weight::Int
    m::Int
    i::Function
    n::Function
    latex_repr::String

    function ISymb(args::Vararg{Int})
        # Validate the number of arguments
        if length(args) % 2 == 0 || length(args) < 3
            throw(ArgumentError("The number of arguments should be odd and at least 3"))
        end

        m = div(length(args), 2) - 1
        i(r) = r > 0 ? args[1 + 2 * r] : 0
        n(r) = args[2 + 2 * r]
        argstrs = String[]

        # Check the validity of arguments
        if m < 0 || any(n(r) < 1 || i(r) >= i(r + 1) for r in 1:m)
            throw(ArgumentError("The arguments are not valid"))
        end

        # Form the latex string
        for r in 0:m
            push!(argstrs, "a_{$(i(r))}")
            if n(r) == 2
                push!(argstrs, "0")
            elseif n(r) > 2
                push!(argstrs, "0^{$(n(r)-1)}")
            end
        end
        push!(argstrs, "a_{$(i(m+1))}")

        # Return the final object
        return new(args, sum(n(r) for r in 0:m) - 1, m, i, n, "I($(join(argstrs, ',')))")
    end
end

Base.isless(a::ISymb, b::ISymb) = begin
    if a.weight != b.weight
        return a.weight < b.weight
    end

    for r in 0: a.m-1
        if a.i(a.m-r) != b.i(b.m-r)
            return a.i(a.m-r) < b.i(b.m-r)
        elseif a.n(a.m-r) != b.n(b.m-r)
            return a.n(a.m-r) > b.n(b.m-r)
        end
    end

    return a.i(0) < b.i(0)
end

struct HSymb <: Symb
    args::Tuple{Vararg{Int}}
    weight::Int
    d::Int
    i::Tuple{Vararg{Int}}
    m::Function
    n::Function
    latex_repr::String

    function HSymb(args::Vararg{Int})
        # Validate the number of arguments
        if length(args) % 2 == 0 || length(args) < 3
            throw(ArgumentError("The number of arguments should be odd and at least 3"))
        end

        d = div(length(args), 2)
        i = cumsum(args[1:2:end])
        m(r) = args[2 * r - 1]
        n(r) = args[2 * r]
        weight = n(1)==0 ? 1 : sum(n(r) for r in 1:d)
        argstrs = String[]
        indices = Int64[]

        # Check the validity of arguments
        if d <= 0 || any(m(r) < 1 for r in 1:d+1) || n(1)<0 || (d > 1 && any(n(r) < 1 for r in 1:d))
            throw(ArgumentError("The arguments are not valid"))
        end

        # Form the latex string
        for r in 1:d
            push!(argstrs, "x_{$(i[r])\\to $(i[r+1])}")
            push!(indices, n(r))
        end

        # Return the final object
        return new(args, weight, d, i, m, n, "[$(join(argstrs, ','))]_{$(join(indices, ','))}")
    end
end

Base.isless(a::HSymb, b::HSymb) = begin
    if a.weight != b.weight
        return a.weight < b.weight
    elseif a.i[end] != b.i[end]
        return a.i[end] < b.i[end]
    end

    for r in 0: a.d-1
        if a.m(a.d-r) != b.m(b.d-r)
            return a.m(a.d-r) > b.m(b.d-r)
        elseif a.n(a.d-r) != b.n(b.d-r)
            return a.n(a.d-r) > b.n(b.d-r)
        end
    end

    return false
end

function partial_differential(H::HSymb, r::Int)
    if r > H.d || r < 1
        throw(ArgumentError("The partial differential is invalid"))
    end
end

function differential(H::HSymb)

end

function Phi(I::Union{Number, ISymb, monomial{ISymb}, polynomial{ISymb}}, d::Int)
    if I isa Number
        return I
    elseif I isa monomial{ISymb}
        return prod(map(x -> Phi(x.first, d)^(x.second), I.args))
    elseif I isa polynomial{ISymb}
        return I.intercept + sum(map(x -> x.second * Phi(x.first, d), I.args))
    end

    # check i_{m+1} <= d + 1
    if d + 1 < I.i(I.m+1)
        throw(ArgumentError("The depth is invalid"))
    end

    if I.m == 0 && I.n(1) == 1
        return 1

    elseif I.i(0) == 0 && I.i(I.m+1) == 0
        return 0

    elseif I.i(0) > 0 && I.i(I.m+1) == 0
        return (-1)^(I.weight) * Phi(ISymb(reverse(I.args)), d)

    elseif I.i(0) > 0 && I.i(I.m+1) > 0
        return sum(
            sum(
                Phi(ISymb(I.args[1:2*k+1]..., p, 0), d) * Phi(ISymb(0, I.n(k)-p, I.args[2*k+3:end]...), d)
                for p in 0:I.n(k)
            )
            for k in 0:I.m
        )

    elseif I.i(0) == 0 && I.i(I.m+1) == d + 1
        return sum(
            (-1)^(I.n(0)+I.m-1) * prod(binomial(I.n(r)+p[r]-1, p[r]) for r in 1:I.m)
            * HSymb(I.i(1), vcat(([I.n(r)+p[r], I.i(r+1)-I.i(r)] for r in 1:I.m)...)...)
            for p in ordered_partitions(I.n(0)-1, I.m; positive = false)
        )

    elseif I.i(0) == 0 && I.i(I.m+1) > 0
        return sum(
            sum(
                (-1)^(I.n(0)+p0+I.m-1) * HSymb(I.i(I.m+1),1,d+1)^p0 * prod(binomial(I.n(r)+p[r]-1, p[r]) for r in 1:I.m)
                * HSymb(I.i(1), vcat(([I.n(r)+p[r], I.i(r+1)-I.i(r)] for r in 1:I.m)...)...)
                for p in ordered_partitions(I.n(0)-1-p0, I.m; positive = false)
            )
            for p0 in 0:I.n(0)-1
        )
    end
end

function Phi_inv(H::Union{Number, HSymb, monomial{HSymb}, polynomial{HSymb}})
    if H isa Number
        return I
    elseif H isa monomial{HSymb}
        return prod(map(x -> Phi_inv(x.first, d)^(x.second), H.args))
    elseif H isa polynomial{HSymb}
        return H.intercept + sum(map(x -> x.second * Phi_inv(x.first, d), H.args))
    else
        return (-1)^H.d * ISymb(0, 1, vcat(([H.i[r], H.n(r)] for r in 1:H.d)...)..., H.i[end])
    end
end



function fundamental_column(h::HSymb)
    visited = Set{HSymb}()
    result = []

    function dfs(H::HSymb)
        if H in visited
            return
        end

        push!(result, Phi_inv(H))
        push!(visited, H)

        if H.d == 1 && H.n(1) == 1
            return
        end

        for r in 1:H.d
            if H.n(r) > 1
                dfs(HSymb(H.args[1:2*r-1]..., H.n(r)-1, H.args[2*r+1:end]...))
            elseif r == H.d
                dfs(HSymb(H.args[1:end-3]..., H.m(H.d)+H.m(H.d+1)))
            else
                dfs(HSymb(H.args[1:2*r-2]..., H.m(r)+H.m(r+1), H.args[2*r+2:end]...))
                dfs(HSymb(H.args[1:2*r-1]..., H.n(r+1), H.m(r+1)+H.m(r+2), H.args[2*r+4:end]...))
            end
        end
    end

    dfs(h)
    return [1, sort(result)...]
end



end

Main.LogToolKit

In [2]:
# include("LogToolKit.jl")
using .LogToolKit
using LaTeXStrings
using Latexify
using Combinatorics

## $\mathbb I^{\text{Symb}}$

$I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{n_m-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})$ is of weight $n_0+\cdots+n_m-1$

There is a total ordering $(i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})\prec(i'_0, n'_0, i'_1, n'_1, \cdots, i'_{m'}, n'_{m'}, i'_{m'+1})$
- if: $\sum n_k < \sum n'_k$
<!-- - else if: $i_{m+1} < i'_{m'+1}$ -->
- else: $i_{m-r} < i'_{m'-r}$ or $n_{m-r} > n'_{m'-r}$ and the entries to the right coincide

### $\Phi$ map

- $\Phi(I(a; b)) = 1,\quad \Phi(I(0; \cdots; 0)) = 0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};0))=(-1)^{n_0+\cdots+n_m-1}\Phi(I(0;0^{n_m-1},a_{i_m},\cdots,0^{n_0-1};a_{i_0})),\quad a_{i_0}\neq0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))\\
\displaystyle=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(I(a_{i_0};\cdots,a_{i_k},0^{p-1};0))\Phi(I(0,0^{q-1},a_{i_{k+1}},\cdots;a_{i_{m+1}})),\quad a_{i_0}, a_{i_{m+1}} \neq 0$

- $\Phi(I(0;0^{n_0-1},a_{i_1},0^{n_1-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))=\\
\displaystyle\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{\log^{p_0}(a_{i_{m+1}})}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\left[\frac{a_{i_2}}{a_{i_1}},\cdots,\frac{a_{i_{m+1}}}{a_{i_m}}\right]_{n_1+p_1,\cdots,n_m+p_m}$

Note: $\log^0(a_{i_{m+1}})$ is taken to be $1$ for $p_0=0$ even if $a_{i_{m+1}}=1$

## $\mathbb H^{\text{Symb}}$

$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow(i_1,n_1,i_2-i_1,\cdots,i_d-i_{d-1},n_d,i_{d+1}-i_d)
$$

Or $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ so that $i_r = m_1 + \cdots + m_r$

The total ordering is $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\prec(m'_1,n'_1,\cdots,m'_{d'},n'_{d'},m'_{d'+1})$
- if: $\sum n_k < \sum n'_k$
- else if: $\sum m_k < \sum m'_k$
- else: $m_{d-r} > m'_{d'-r}$ or $n_{d-r} > n'_{d'-r}$ and the entries to the right coincide

The $\partial_{r}$ of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ is
- if $d = 1$ and $n_1 = 1$: $-dv_{i_1,i_2-1}$
- else if $n_r>1$: $(\cdots,n_r-1,\cdots) du_{i_r, i_{r+1}-1}$
- else if $r=d$: $-(\cdots,n_{d-1},m_d+m_{d+1}) dv_{i_d,i_{d+1}-1}$
- else: $-(\cdots,m_{r}+m_{r+1},n_{r+1},\cdots) dv_{i_r, i_{r+1}-1} + (\cdots,m_r,n_{r+1},m_{r+1}+m_{r+2},\cdots) (du_{i_r, i_{r+1}-1} - dv_{i_r, i_{r+1}-1})$

Since
$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow I(a_0;a_{i_1},0^{n_1-1},\cdots,a_{i_d},0^{n_d-1};a_{i_{d+1}})
$$
so $\Phi^{-1}$ is implemented as
$$
(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\rightsquigarrow(0,1,i_1,n_1,\cdots,i_d,n_d,i_{d+1})
$$

## Implementation of $\Phi$

- $\Phi(i_0,1,i_1) = 1,\quad \Phi(0,n_0,\cdots,n_m,0) = 0$

- $\Phi(i_0, n_0, \cdots, i_m, n_m, 0)=(-1)^{n_0+\cdots+n_m-1}\Phi(0,n_m,i_m,\cdots,n_0,i_0),\quad i_0\neq0$

- $\displaystyle\Phi(i_0,n_0,\cdots,i_m,n_m,i_{m+1})=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(i_0,\cdots,i_k,p;0)\Phi(0,q,i_{k+1},\cdots;i_{m+1}),\quad i_0,i_{m+1} \neq 0$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,d+1)=\sum_{p_1+\cdots+p_m=n_0-1}(-1)^{n_0+m-1}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,d+1-i_m)$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,i_{m+1})=\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{(i_{m+1},1,d+1)^{p_0}}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,i_{m+1}-i_m)$

In [5]:
# tests

LaTeXString((
    Phi(ISymb(0, 6, 1, 2, 3, 3, 6, 4, 10) + 6* ISymb(0,1,1,1,2)^2 + 4, 10)
).latex_repr)

"(4)+(56)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,9}+(105)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,4,8}+(70)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{3,3,8}+(120)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,5,7}+(120)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{3,4,7}+(60)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{4,3,7}+(100)[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,6" ⋯ 2760 bytes ⋯ "_{10\\to 21}]_{1}^{3}[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{4,3,4}+(4)[x_{10\\to 21}]_{1}^{4}[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,5}+(3)[x_{10\\to 21}]_{1}^{4}[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,4,4}+(2)[x_{10\\to 21}]_{1}^{4}[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{3,3,4}+(-1)[x_{10\\to 21}]_{1}^{5}[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,4}"

## $\mathbb Q$ algberas

In [3]:
# tests

println((
    4 * ISymb(0,1,1,1,2)
).latex_repr)
println((
    4 * monomial{Symb}(ISymb(0,1,1,1,2), (ISymb(0,1,1,3,2), 3))
).latex_repr)
println((
    4 * polynomial{Symb}(monomial{Symb}(ISymb(0,1,1,1,2)))
).latex_repr)
println((
    ISymb(0,1,1,1,2) * ISymb(0,1,1,1,2)
).latex_repr)
println((
    (4*ISymb(0,1,1,3,2)) + (2*ISymb(0,1,1,1,2))
).latex_repr)
println((
    (4*ISymb(0,1,1,3,2)) + (2*ISymb(0,1,1,1,2)^7) - 4 * ISymb(0,1,1,5,2)
).latex_repr)

(4)I(a_{0},a_{1},a_{2})
(4)I(a_{0},a_{1},a_{2})I(a_{0},a_{1},0^{2},a_{2})^{3}
(4)I(a_{0},a_{1},a_{2})
I(a_{0},a_{1},a_{2})^{2}
(2)I(a_{0},a_{1},a_{2})+(4)I(a_{0},a_{1},0^{2},a_{2})
(4)I(a_{0},a_{1},0^{2},a_{2})+(-4)I(a_{0},a_{1},0^{4},a_{2})+(2)I(a_{0},a_{1},a_{2})^{7}


## $\mathbb L^{\text{Symb}}$

## Tensor algebra

$$
\left(\sum_{i_1}c_{i_i}^{(1)}t_{i_1}^{(1)}\right)\otimes\cdots\otimes\left(\sum_{i_N}c_{i_N}^{(N)}t_{i_N}^{(N)}\right)=\sum_{i_1,\cdots,i_N}c_{i_i}^{(1)}\cdots c_{i_N}^{(N)} t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(N)}
$$

$$
\left(\sum_{\mathbf i}c_{\mathbf i}^{(1)}t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(1)}\right)\left(\sum_{\mathbf j}c_{\mathbf j}^{(2)}t_{j_1}^{(2)}\otimes\cdots\otimes t_{j_N}^{(2)}\right)=\sum_{\mathbf i,\mathbf j}c_{\mathbf i}^{(1)}c_{\mathbf j}^{(2)}\left(t_{i_1}^{(1)}t_{j_1}^{(2)}\right)\otimes\cdots\otimes\left(t_{i_N}^{(1)}t_{j_N}^{(2)}\right)
$$

## $\bigwedge^*\mathbb L^{\text{Symb}}$

In [ ]:
function complementary_entry(H1::Union{HSymb, Int}, H2::Union{HSymb, Int})
    k, l = H1.d, H2.d
    i(r) = r == 0 ? 0 : H1.i[r]
    j(r) = r == 0 ? 0 : H2.i[r]
    m, p = H1.m, H2.m
    overlapping_indices = Int[]
    for r in 1:k
        index = findfirst(x->x==i(r), map(j, ))
    end


    function Isigma(r::Int)
        if m(r) ==1
            return
        end
        return sum(
            sum(
                ISymb(j(q(r)),...,j(s),u,0) *
                ISymb(0,v,j(s+1),...,j(q(r+1)))
                for u in 0:p(s)-m(r)
            )
            for s in q(r):q(r+1)-1 if m(r) <= p(s)
        )
    end

    return (-1)^(l-k) * prod(Isigma(r) for r in 0:k)
end

## Variation matrix over $S_d(\mathbb C)$

Recall the *complementary entry* of $(-1)^kI(0;a_{i_1},0^{m_{1}-1},\cdots,a_{i_k},0^{m_{k}-1};1)$ with respect to $(-1)^lI(0;a_{j_1},0^{p_{1}-1},\cdots,a_{j_l},0^{p_{l}-1};1)$ is
$$
(-1)^{l-k} I^{\sigma_{i_1}\sigma_0^{m_{1}-1}\cdots\sigma_{i_k}\sigma_0^{m_{k}-1}}(0;a_{j_1},0^{p_{1}-1},\cdots,a_{j_l},0^{p_{l}-1};1)
$$

$\sigma_{i_1}\sigma_0^{m_1-1}\cdots\sigma_{i_k}\sigma_0^{m_k-1}$ should correspond to $(0,1,i_1,m_1,\cdots, i_k,m_k)$

Suppose $\{i_1,\cdots,i_k\}=\{j_{q_1},\cdots,j_{q_k}\}$ is a subsequence of $\{j_1,\cdots,j_l\}$ (otherwise the complementary entry is 0), then the complementary entry is

$$
(-1)^{l-k}I(0;\cdots;a_{j_{q_1}})\left(\prod_{r=1}^{k-1} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})\right)I^{\sigma_0^{m_k-1}}(a_{j_{q_k}};\cdots;1)
$$

Assume
- $a_0=0$, $a_{d+1}=1$
- $i_0=j_0=0$, $i_{k+1}=j_{l+1}=d+1$ so that $q_0=0$, $q_{k+1}=l$,
- $m_0=1$

then the complementary entry is
$$
(-1)^{l-k}\prod_{r=0}^{k} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})
$$
If $m_r > 1$, it is equal to
$$
I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})=\sum_{\substack{q_r\leq s<q_{r+1} \\ m_r \leq p_s}}\sum_{u+v=p_s-m_r}I(a_{j_{q_r}};\cdots,a_{j_s},0^{u};0)I(0;0^{v},a_{j_{s+1}},\cdots;a_{j_{q_{r+1}}})
$$



`complementaryEntry(w1, w2)`: the complementary entry of $w_1 = (0,1,j_1,p_1,\cdots,j_l,p_l)$ and $w_2 = (0,1,i_1,m_1,\cdots,i_k,m_k)$. Which should be the product of `Isigma(r)` from $r=0$ to $k$

In [36]:
# map(x -> x isa Number ? x : x.latex_repr, fundamental_column(HSymb(1,2,2,3,3,4,4)))
map(
    x -> x isa Number ? x : Phi(x, 10).latex_repr,
    fundamental_column(HSymb(1,2,2,3,3,4,4))
)

69-element Vector{Any}:
 1
  "[x_{1\\to 10}]_{1}"
  "[x_{3\\to 10}]_{1}"
  "[x_{6\\to 10}]_{1}"
  "[x_{1\\to 10}]_{2}"
  "[x_{3\\to 10}]_{2}"
  "[x_{1\\to 3},x_{3\\to 10}]_{1,1}"
  "[x_{6\\to 10}]_{2}"
  "[x_{1\\to 6},x_{6\\to 10}]_{1,1}"
  "[x_{3\\to 6},x_{6\\to 10}]_{1,1}"
  "[x_{1\\to 10}]_{3}"
  "[x_{3\\to 10}]_{3}"
  "[x_{1\\to 3},x_{3\\to 10}]_{1,2}"
 ⋮
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,1}"
  "[x_{1\\to 6},x_{6\\to 10}]_{3,4}"
  "[x_{3\\to 6},x_{6\\to 10}]_{3,4}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{1,2,4}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,1,4}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{1,3,3}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,2,3}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,2}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{1,3,4}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,2,4}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,3}"
  "[x_{1\\to 3},x_{3\\to 6},x_{6\\to 10}]_{2,3,4}"

## One form

## Connection form